このノートブックはDay3任意課題の実験に対応したノートブックです。

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 演習環境の準備

In [2]:
%pip install --upgrade pip
%pip install --upgrade transformers
# %pip install google.colab
%pip install google-colab-selenium
%pip install bitsandbytes
%pip install -U openai
%pip install ipywidgets
%pip install accelerate
%pip install huggingface_hub

In [3]:
from pickle import TRUE

# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login(TRUE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'notebook_login': pass new_session=b'I01\n' as keyword args. From version 1.0 passing these as positional arguments will result in an error,
  warnings.warn(


In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random

random.seed(0)

In [6]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # device_map="cpu",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float32
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### ベースラインモデル評価

In [7]:
def generate_output(query):
    messages = [
        {"role": "user", "content": query},
    ]
    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=False,
        # temperature=0.6, # If do_sample=True
        # top_p=0.9,  # If do_sample=True
    )

    response = outputs[0][input_ids.shape[-1] :]
    return tokenizer.decode(response, skip_special_tokens=True)

In [8]:
questions = [
    "ジョブクラウンの社長は誰ですか？",
    "ジョブクラウンの企業理念は何ですか？",
    "入社初日に閲覧すべき社内規程やマニュアルはどこにありますか？",
    "交通費と通勤費の違いを教えてください。",
    "週報や日報で記載するべき内容を教えてください。",
]

In [9]:
responses = []
for question in questions:
    responses.append((generate_output(question)))

print(responses)

[' ジョブクラウンの社長は**藤井 浩**です。 \n\n\n\n', 'ジョブクラウンの企業理念は「**お客様の夢を叶える**」です。 \n\nこれは、企業が顧客のニーズを理解し、そのニーズに応えることで、お客様の夢を実現することを目指すことを表現しています。\n\n\n \n\n\n', '入社初日に閲覧すべき社内規程やマニュアルは、**会社によって異なります**。 \n\n一般的には、以下の場所から確認できます。\n\n* **人事部:**  入社時に人事担当者に相談して確認できます。\n* **会社 intranet:**  会社内にあるオンラインシステムで、社内規程やマニュアルが掲載されています。\n* **会社ホームページ:**  会社ホームページに、社内規程やマニュアルのリンクが掲載されている場合があります。\n* **入社説明会資料:**  入社説明会で配布された資料に、社内規程やマニュアルの項目が記載されている場合があります。\n* **部署の担当者:**  部署の担当者に直接問い合わせることもできます。\n\n\n \n\n\n \n\n\n', '## 交通費と通勤費の違い\n\n**交通費**と**通勤費**はどちらも、**移動にかかる費用**ですが、**目的や範囲が異なります**。\n\n**交通費**は、**一般的に移動手段の利用費用**を指します。\n* **電車、バス、タクシー、飛行機、船**など、様々な手段で移動する際に支払う費用です。\n* **目的地や距離、利用頻度によって金額が異なります**。\n\n**通勤費**は、**仕事や学校に通うために必要な移動費用**を指します。\n* **主に自宅から職場や学校までの移動**を指します。\n* **交通手段や距離、通勤時間によって費用が異なります**。\n\n**簡単にまとめると:**\n\n* **交通費:**  一般的に移動手段の利用費用\n* **通勤費:**  仕事や学校に通うために必要な移動費用\n\n\n \n\n\n \n', '## 週報や日報で記載すべき内容\n\n週報や日報は、チームの進捗状況や課題を明確に伝えるための重要なツールです。 \n内容を明確にすることで、チームメンバーとコミュニケーションを円滑に行い、より良い成果を上げるための基盤を築くことができま

In [10]:
responses[0]

' ジョブクラウンの社長は**藤井 浩**です。 \n\n\n\n'

### 評価実装

In [11]:
gold_answer = [
    "反町 好克",
    "人を通じより良い世界を",
    "https://jobcrown.backlog.com/projects/GUIDANCE",
    "通勤費は「自宅～現場間の交通費用」である。交通費は通勤費以外である。",
    "週報や日報で記載するべき内容は、テンプレートでは次の通りである。「プロジェクト概要」「提出期限」「所感・感想」「目標と結果」「未達理由」「次週の目標」「所感（自由記述欄）」。\n"
    "ただしこの限りではない。",
]

In [12]:
gold_answer[0]

'反町 好克'

In [13]:
# @title 評価実装

from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)


def openai_generator(query):

    messages = [{"role": "user", "content": query}]

    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
        "Instruction: You are a world class state of the art assistant for rating "
        "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
        "Say 4, if User Answer is full contained and equivalent to Reference Answer"
        "in all terms, topics, numbers, metrics, dates and units.\n"
        "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
        "in all terms, topics, numbers, metrics, dates and units.\n"
        "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
        "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
        "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
        "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
        "### Question: {query}\n"
        "### {answer0}: {sentence_inference}\n"
        "### {answer1}: {sentence_true}\n"
        "The rating is:\n"
    )
    template_accuracy2 = (
        "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
        "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
        "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
        "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
        "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
        "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
        "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
        "Question: {query}\n\n"
        "{answer0}: {sentence_inference}\n\n"
        "{answer1}: {sentence_true}\n\n"
        "Rating: "
    )

    score1 = openai_generator(
        template_accuracy1.format(
            query=query,
            answer0="User Answer",
            answer1="Reference Answer",
            sentence_inference=response,
            sentence_true=reference,
        )
    )
    try:
        score1 = int(score1)
    except:
        print("Failed")
        score1 = 0

    score2 = openai_generator(
        template_accuracy2.format(
            query=query,
            answer0="Reference Answer",
            answer1="User Answer",
            sentence_inference=reference,
            sentence_true=response,
        )
    )

    try:
        score2 = int(score2)
    except:
        print("Failed")
        score2 = 0

    return (score1 + score2) / 2

In [14]:
# 評価
scores = []
for i in range(len(questions)):
    score = evaluate_answer_accuracy(questions[i], responses[i], gold_answer[i])
    scores.append(score)

print(scores)

[0.0, 0.0, 0.0, 1.0, 1.0]


### RAG実装

In [15]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

In [16]:
%pip install langchain_openai

In [17]:
import os
import glob
import re
import json
from pathlib import Path


def extract_sentences(text):
    """テキストを文単位に分割する"""
    # 日本語の文の区切りを考慮
    sentences = re.split(r".+?[。！？](?=\s|$)", text)
    # 空の文を削除
    return [s.strip() for s in sentences if s.strip()]


def process_files(directories):
    """指定されたディレクトリ内のすべてのtxtファイルを処理"""
    all_sentences = []

    for directory in directories:
        txt_files = glob.glob(os.path.join(directory, "*.txt"))

        for file_path in txt_files:
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()

                # ファイル名を取得（パスなし）
                file_name = os.path.basename(file_path)

                # 文に分割
                sentences = extract_sentences(content)

                # メタデータを追加
                for sentence in sentences:
                    if sentence:  # 空でない文のみ
                        all_sentences.append(
                            {
                                "source_file": file_name,
                                "content": sentence,
                                "path": file_path,
                            }
                        )
            except Exception as e:
                print(f"ファイル処理エラー {file_path}: {e}")

    return all_sentences

In [18]:
# 処理するディレクトリ
directories = [
    "/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki",
    "/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki2",
]

# 文を抽出
sentences = process_files(directories)

# 結果を保存
with open("rag_sentences.json", "w", encoding="utf-8") as f:
    json.dump(sentences, f, ensure_ascii=False, indent=2)

print(f"処理完了: {len(sentences)}文を抽出しました")

処理完了: 942文を抽出しました


In [19]:
def load_rag_sentences(json_path="rag_sentences.json"):
    """
    rag_sentences.jsonを読み込み、リストとして返す

    Args:
        json_path: JSONファイルのパス

    Returns:
        文のリスト
    """
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            sentences_data = json.load(f)

        print(f"読み込み完了: {len(sentences_data)}文を読み込みました")
        return sentences_data

    except Exception as e:
        print(f"ファイル読み込みエラー: {e}")
        return []


def process_sentences(sentences_data):
    """
    文のリストを処理する例

    Args:
        sentences_data: 文のリスト
    """
    # 例: 各文を表示
    for i, item in enumerate(sentences_data[:5]):  # 最初の5つだけ表示
        print(f"文 {i+1}:")
        print(f"  ソース: {item['source_file']}")
        print(
            f"  内容: {item['content'][:50]}..."
            if len(item["content"]) > 50
            else f"  内容: {item['content']}"
        )
        print(f"  パス: {item['path']}")
        print("-" * 50)

    # 例: 特定のキーワードを含む文を検索
    keyword = "勤怠"
    matching_sentences = [item for item in sentences_data if keyword in item["content"]]
    print(f"'{keyword}'を含む文: {len(matching_sentences)}件")

    # 例: ソースファイル別に文をグループ化
    files_count = {}
    for item in sentences_data:
        source = item["source_file"]
        if source in files_count:
            files_count[source] += 1
        else:
            files_count[source] = 1

    print("\nファイル別の文の数:")
    for file, count in list(files_count.items())[:5]:  # 最初の5つだけ表示
        print(f"  {file}: {count}文")

    return sentences_data

In [20]:
# JSONファイルを読み込む
sentences_data = load_rag_sentences()

if sentences_data:
    # 文のリストを処理
    sentence = process_sentences(sentences_data)

# ここに独自の処理を追加できます
# 例: RAGモデルのための埋め込み生成など

読み込み完了: 942文を読み込みました
文 1:
  ソース: 2841766_セキュリティ関連.txt
  内容: # セキュリティ関連
# ID: 2841766
# 作成者: 赤坂 百夏
# 作成日時: 2023...
  パス: /content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt
--------------------------------------------------
文 2:
  ソース: 2841766_セキュリティ関連.txt
  内容: ##誓約書
  パス: /content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt
--------------------------------------------------
文 3:
  ソース: 2841766_セキュリティ関連.txt
  内容: ##セキュリティ研修について
  パス: /content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt
--------------------------------------------------
文 4:
  ソース: 2840473_空き時間にやってほしいこと.txt
  内容: # 空き時間にやってほしいこと
# ID: 2840473
# 作成者: 赤坂 百夏
# 作成日時:...
  パス: /content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2840473_空き時間にやってほしいこと.txt
--------------------------------------------------
文 5:
  ソース: 2840473_空き時間にやってほしいこと.txt
  内容: ##対応（優先度順）

1、slack、backlogのアプリダウンロー

In [21]:
with open("/content/rag_sentences.json", "r") as f:
    raw_writedown = f.read()

In [22]:
# ドキュメントを用意する。
documents = sentence
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[10])

ドキュメントサイズ:  942
ドキュメントの例: 
 {'source_file': '2840299_基本ルール.txt', 'content': '##研修中の勤怠連絡について', 'path': '/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2840299_基本ルール.txt'}


In [23]:
documents[10]["content"]

'##研修中の勤怠連絡について'

In [24]:
similarity_scores = []
# document_embeddings = []
# Retrievalの実行
for i, question in enumerate(questions):
    query_embeddings = emb_model.encode([question], prompt_name="query")

    for doc in documents:
        content = doc["content"]
        document_embeddings = emb_model.encode(content)

        # 各ドキュメントの類似度スコア
        similarity_score = (query_embeddings @ document_embeddings.T) * 100
        similarity_scores.append((i, doc, similarity_score))

print(similarity_scores)

[(0, {'source_file': '2841766_セキュリティ関連.txt', 'content': '# セキュリティ関連\n# ID: 2841766\n# 作成者: 赤坂 百夏\n# 作成日時: 2023-07-27T02:00:55Z\n# 更新日時: 2023-07-31T09:27:52Z\n\n#情報セキュリティ読本の読み合せと誓約書の記入\n\n\n\n##情報セキュリティ読本', 'path': '/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt'}, array([58.08872], dtype=float32)), (0, {'source_file': '2841766_セキュリティ関連.txt', 'content': '##誓約書', 'path': '/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt'}, array([57.52232], dtype=float32)), (0, {'source_file': '2841766_セキュリティ関連.txt', 'content': '##セキュリティ研修について', 'path': '/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt'}, array([59.339523], dtype=float32)), (0, {'source_file': '2840473_空き時間にやってほしいこと.txt', 'content': '# 空き時間にやってほしいこと\n# ID: 2840473\n# 作成者: 赤坂 百夏\n# 作成日時: 2023-07-26T09:01:41Z\n# 更新日時: 2025-03-31T06:58:26Z\n\n#空き時間にやってほしいこと', 'path': '/content/drive/MyDrive/松尾研AIエンジニアリング

In [25]:
similarity_scores[0]

(0,
 {'source_file': '2841766_セキュリティ関連.txt',
  'content': '# セキュリティ関連\n# ID: 2841766\n# 作成者: 赤坂 百夏\n# 作成日時: 2023-07-27T02:00:55Z\n# 更新日時: 2023-07-31T09:27:52Z\n\n#情報セキュリティ読本の読み合せと誓約書の記入\n\n\n\n##情報セキュリティ読本',
  'path': '/content/drive/MyDrive/松尾研AIエンジニアリング実践講座/Day3_機械学習システムの改善/backlog_wiki/2841766_セキュリティ関連.txt'},
 array([58.08872], dtype=float32))

In [26]:
topk = 5
after_rag_scores = []

for i, question in enumerate(questions):
    print(question)

    # Get similarity scores for the current question
    current_question_scores = [
        (q_idx, doc, score) for q_idx, doc, score in similarity_scores if q_idx == i
    ]
    # Extract only the similarity score values
    flatten_scores = [
        score.item() for _, _, score in current_question_scores
    ]  # Extract similarity score values

    # flatten_scores = similarity_scores[i][0]
    # print(flatten_scores)
    # print(flatten_scores.argsort()[i:-1])
    sorted_scores_indices = sorted(
        range(len(flatten_scores)), key=lambda k: flatten_scores[k], reverse=True
    )

    topk_indices = sorted_scores_indices[:topk]
    for k, index_in_scores in enumerate(topk_indices):
        index_in_documents = current_question_scores[index_in_scores][
            1
        ]  # get document from current_question_scores
        # print(f"取得したドキュメント{k+1}: (Similarity_Score: {flatten_scores[index_in_scores]})")
        # print(index_in_documents['content'], "\n\n")

    references = "\n".join(
        [
            "* " + documents[index_in_documents]["content"]
            for index_in_documents in topk_indices
        ]
    )
    query = f"\n\n[質問] {question} \n\n 以下の文書の内容に基づいて、事実としてわかる範囲でのみ答えてください。\n\n [参考資料]\n {references}"

    # print(query)

    response = generate_output(query)
    print(response)

    after_rag_score = evaluate_answer_accuracy(questions[i], response, gold_answer[i])
    after_rag_scores.append(after_rag_score)

print(after_rag_scores)

ジョブクラウンの社長は誰ですか？
ジョブクラウンの社長は記載されていません。



ジョブクラウンの企業理念は何ですか？
参考資料からジョブクラウンの企業理念を特定することはできません。



入社初日に閲覧すべき社内規程やマニュアルはどこにありますか？
回答：参考資料には、入社初日に閲覧すべき社内規程やマニュアルの場所が記載されていません。



交通費と通勤費の違いを教えてください。
## 交通費と通勤費の違い

**通勤費**は、自宅から現場までの交通費用です。

**交通費**は、自宅から現場以外の場所（例：現場～本社、自宅～面談先など）までの交通費用です。


 



週報や日報で記載するべき内容を教えてください。
日報・週報のどちらで提出するべきかについては、**休日出勤の場合は備考欄へ代休日or残業計上いずれかを記載**、**代休の場合は備考欄へ休日出勤日を記載**します。 




[0.0, 0.0, 0.0, 3.0, 0.0]
